Example of dataloader usage. Unsure of how to load skin segmentation data at this time.

In [ ]:
# Datasest dir should look like:


# ├── data
# │   ├── v1_cam1_no_split
# |      ├── Adjust Radio
# |          ├── 1.jpg
# │      ├── Drink
# |           ├── 1.jpg
# |      ├── Test_data_list.csv
# |      ├── Train_data_list.csv
# |
# │   ├── v2_cam1_cam2_ split_by_driver
# |      ├── Camera 1
# |          ├── test
# |          ├── train
# │      ├── Camera 2
# |          ├── test
# |          ├── train
# |      ├── skin_nonskin_pixels


Example on how to (separately) export detector and cnns models to ONNX formats 

In [5]:
from cnn.hands_cnn import Hands_VGG16
from wrappers.hands_wrapper import *
import torch.onnx
import argparse


args = argparse.ArgumentParser()
args.add_argument('--model', type=str, default='hands_vgg')
args.add_argument('--distributed', type=bool, default=False)

args.add_argument('--resume_path', type=str, default=None)
args.add_argument('--resume_last_epoch', type=bool, default=False)

args.add_argument('--hidden_units', type=int, default=128)
args.add_argument('--freeze', type=bool, default=True)
args.add_argument('--batch_size', type=int, default=64)
args.add_argument('--epochs', type=int, default=30)

args.add_argument('--lr', type=float, default=1e-3)
args.add_argument('--num_frozen_params', type=int, default=30)
args.add_argument('--dropout', type=float, default=0.5)
args.add_argument('--optimizer', type=str, default='SGD')
args.add_argument('--weight_decay', type=float, default=0.0)
args.add_argument('--scheduler', action='store_true')

args.add_argument('--save_period', type=int, default=5)
args.add_argument('--transform', type=bool, default=True) 
args.add_argument('--device', type=str, default='cuda')

args.add_argument('--data_dir', type=str, default='data/v2_cam1_cam2_split_by_driver')
args.add_argument('--model_dir', type=str, default='cnn/hands_models')
args.add_argument('--detector_path', type=str, default='path/to/yolo/weights')

args = args.parse_args()

hands_model = Hands_VGG16(args, 10).to('cuda')
x = torch.randn(64, 3, 224, 224).to('cuda')

hands_model.load_state_dict(torch.load("/home/ron/Classes/CV-Systems/cybertruck/cnn/hands_models/vgg/epoch60_11-16_03:44:44.pt"))
torch.onnx.export(hands_model, x, 'hands_cnn.onnx', export_params=True, opset_version=17, do_constant_folding=True, input_names = ['input'], 
                  output_names = ['output'], dynamic_axes={'input' : {0 : 'batch_size'}, 'output' : {0 : 'batch_size'}})


In [6]:
from ultralytics import YOLO

hands_detector = YOLO('detection/hands_detection/runs/detect/best/weights/best.pt')
hands_detector.export(format='onnx')

Ultralytics YOLOv8.0.208 🚀 Python-3.10.13 torch-2.1.0+cu121 CPU (12th Gen Intel Core(TM) i7-12700H)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'detection/hands_detection/runs/detect/best/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (6.0 MB)

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 0.5s, saved as 'detection/hands_detection/runs/detect/best/weights/best.onnx' (11.7 MB)

Export complete (2.2s)
Results saved to /home/ron/Classes/CV-Systems/cybertruck/detection/hands_detection/runs/detect/best/weights
Predict:         yolo predict task=detect model=detection/hands_detection/runs/detect/best/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=detection/hands_detection/runs/detect/best/weights/best.onnx imgsz=640 data=/home/ron/Classes/CV-Systems/cybertruck/detection/hands_detection/egohands.yaml  
Visualize:       https://netron.app


'detection/hands_detection/runs/detect/best/weights/best.onnx'